<a href="https://colab.research.google.com/github/bfdefraga/projetos-portfolio-/blob/main/Projeto_COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ANÁLISE EXPLORATÓRIA DE DADOS: COVID-19**

# 1. Contextualização

A pandemia de COVID-19 impactou fortemente os sistemas de saúde pelo mundo causando também diversos prejuízos econômicos e sociais. A COVID-19 é uma infecção respiratória aguda causada pelo vírus coronavírus SARS-CoV-2 podendo acarretar em sérias complicações respiratórias que podem evoluir para o óbito [link](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus.).

Nesse projeto, explorou-se os dados relativos ao número de casos, de mortes e de vacinados ainda no auge da pandemia nos dando a dimensão da realizade vivida naquele momento no Brasil. Os resultados dessa análise podem ser explorados e visualizados de forma interativa no **Dashboard** disponibilizado nesse [link](https://lookerstudio.google.com/reporting/c9b99be6-9ccf-45df-afd4-91dfa8225f8d/page/zM9xD) do **Looker Studio**.

### 1.1 Origem dos Dados

As informações utilizadas nesse projeto são provenientes de duas importantes organizações de grande reconhecimento internacional e que compilaram os dados provenientes do mundo todo sobre a COVID-19.

Os dados referentes ao número de casos e de óbitos são visualmente disponibilizados pela Universidade John Hopkins e são atualizados diariamente no [link](https://systems.jhu.edu/research/public-health/ncov/). Já os dados utilizados para alimentar essa análise podem ser encontrados nesse [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports).

Por sua vez, os dados relacionados com as informações da vacinação são compilados e disponibilizados para visualização pela Universidade de Oxford via projeto "Our World in Data"  [link](https://ourworldindata.org.). Aqui, os dados para uso podem ser acessados no [link](https://covid.ourworldindata.org/data/owid-covid-data.csv.).

A seguir, a relação das variáveis utilizadas após processamento.

Variáveis relacionadas aos casos:

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país;
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

Variáveis relacionadas à vacinação:

 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

Nas próximas etapas desse projeto, será abordada a fase da análise exploratória dos dados que embasaram a confecção do Dashboard dosponibilizado.

# 2. Análise Exploratória dos Dados

Nessa etapa será apresentado todo o código utilizado para extrair e processar os dados extraídos das fontes indicadas anteriormente e que alimentam o Dashboard criado na plataforma da Google, o Looker Studio.

### 2.1 Bibliotecas e Pacotes Utilizados

In [ ]:
from datetime import datetime, timedelta
import math
import numpy as np
import pandas as pd
from typing import Iterator

### 2.2 Extração e Processamento dos Dados de Casos

### 2.2.1 Extração

Os dados estão compilados em um arquivo por dia.

In [ ]:
#Coleta da base de dados e criação de DataFrame.

cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [ ]:
#Visualização da amostra.

cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


In [ ]:
#Geração de uma lista, por meio do iterador, com todos os dias de 2021.

def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [ ]:
#Definição do período de tempo dos dados a serem utilizados, ano de 2021.

start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

In [ ]:
#Seleção de colunas e linhas referentes ao Brasil.

cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')  #Criação do DataFrame.

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True) #Seleção dos dados do Brasil.
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))  #Criação da coluna data.

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], ignore_index=True)

In [ ]:
#Amostra de estado.

cases.query('Province_State == "Santa Catarina"').head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
23,Santa Catarina,Brazil,494447,5270,6901.069508,2021-01-01
50,Santa Catarina,Brazil,496524,5294,6930.058503,2021-01-02
77,Santa Catarina,Brazil,497345,5314,6941.517321,2021-01-03
104,Santa Catarina,Brazil,498910,5376,6963.360256,2021-01-04
131,Santa Catarina,Brazil,502785,5420,7017.444201,2021-01-05


### 2.2.2. Limpeza e Processamento dos Dados


Nessa fase manipula-se os dados para garantir boa granulidade e qualidade da base de dados com foco na geração do Dashboard.

In [ ]:
#Amostra do Brasil.

cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [ ]:
#Verificar tamanho do banco de dados.

cases.shape

(9828, 6)

In [ ]:
#Visualização geral dos dados.

cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


In [ ]:
#Renomeação de colunas.

cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [ ]:
#Ajuste dos nomes dos estados para manter consistência dos dados georeferenciados.

states_map = {    #ADICIONAMOS ESTADOS COM ACENTO EM FUNÇÃO DO LOOKER TER GEOLOCALIZAÇÃO
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

In [ ]:
#Adição de duas chaves temporais, colunas, para possibilitar o uso de fitros no Looker.

cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
#Cálculo da população estimada por estado por meio da regra de três.

cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

 **NOTA:**

 - A média móvel é uma medida baseada nos números de casos divulgados nos últimos 7 dias dividido por 7. Dessa forma, visa-se diluir o efeito de represamento de dados de um final de semana, por exemplo.

 - Considerando que o tempo de incubação do vírus é de até 14 dias, para avaliar a tendência dos números toma-se a média do dia atual e divide pela média de 7 atrás. Assim, se o valor for igual a 1 indica estabilidade, se maior que 1 indica aumento, e se menor que 1 indica diminuição. Porém, para manter melhor maleabilidade dos dados, considera-se, nesse  projeto, uma margem para cada a tendência a ser explicitada no código abaixo.

In [ ]:
#Criação de função para análise da tendência do número de casos e de mortes.

cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:             #Margem de 0.25 para considerar queda.
    status = 'downward'
  elif rate > 1.15:           #Margem de 0.15 para considerar elevação.
    status = 'upward'
  else:
    status = 'stable'

  return status

In [ ]:
#Separação dos dados por estado a fim de mentar a consistência original e evitar erros de leitura.

for state in cases['state'].drop_duplicates():

  #Seleção por estado e ordenação por data.
  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  #Criação de coluna com operador "diff" para retorno de diferença da coluna anterior.
  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)

  #Aplicação do método "rooling" para selecionar uma janela de 7 dias (7colunas) e método "mean" para gerar a média móvel.
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())

  #Cálculo do valor da tendência com método "shift".
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)

  #Criação de coluna que armazenará a informação de tendência por meio da aplicação do método "get_trend".
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

#Repetição do processo para gerar indicadores sobre o número de mortos.

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

#Retorno das informações por estado.

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = cases_._append(cases_per_state, ignore_index=True)

cases = cases_
cases_ = None

In [ ]:
#Utilização do método ".astype" para garantir consistência dos dados.

cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [ ]:
#Reorganizar colunas.

cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [ ]:
#Amostra do resultado final.

cases.head(n=25)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,2021-01,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,2021-01,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,2021-01,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,2021-01,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,2021-01,2021
5,2021-01-06,Brazil,Acre,881935,42378,208,<NA>,NaN,NaN,814,6,<NA>,NaN,NaN,2021-01,2021
6,2021-01-07,Brazil,Acre,881935,42478,100,<NA>,NaN,NaN,821,7,<NA>,NaN,NaN,2021-01,2021
7,2021-01-08,Brazil,Acre,881935,42814,336,161,NaN,NaN,823,2,4,NaN,NaN,2021-01,2021
8,2021-01-09,Brazil,Acre,881935,42908,94,139,NaN,NaN,823,0,4,NaN,NaN,2021-01,2021
9,2021-01-10,Brazil,Acre,881935,43127,219,155,NaN,NaN,825,2,4,NaN,NaN,2021-01,2021


In [ ]:
#Carregamento/Download do arquivo gerado para uso na plataforma Looker Studio.

cases.to_csv('./covid-cases.csv', sep=',', index=False)

### 2.3 Extração e Processamento dos Dados de Vacinação

### 2.3.1 Extração

In [ ]:
#Extração dos dados.

vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates = [3] )

In [ ]:
#Seleção das colunas de interesse referentes ao Brasil.

vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [ ]:
#Amostra

vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-09


### 2.3.2 Limpeza e processamento dos Dados

Nessa fase manipula-se os dados novamente para garantir boa granulidade e qualidade da base de dados com foco na geração do Dashboard.

In [ ]:
#Amostra

vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-09


In [ ]:
#Verificação do tamanho da base de dados.

vaccines.shape

(1674, 7)

In [ ]:
#Visualização geral dos dados.

vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1674 non-null   object        
 1   population               1674 non-null   int64         
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1674 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 91.7+ KB


In [ ]:
#Utilização do método ".ffillna" para tratamento de dados faltantes, como se não mudasse de um dia para outro nesse caso.

vaccines = vaccines.fillna(method='ffill')

<ipython-input-37-564413049965>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vaccines = vaccines.fillna(method='ffill')


In [ ]:
#Aplicação de filtro a partir da coluna 'date' para garantir que ambas as bases tenham o mesmo período de tempo.

vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

In [ ]:
#Modificação do nome de colunas.

vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

In [ ]:
#Criação de chaves temporais.

vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

In [ ]:
#Geração de dados relativos em % com arredondamento para 4 casas decimais.

vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

In [ ]:
#Aplicação do método ".astype" para manter consistência dos dados.

vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

In [ ]:
#Reorganização das colunas.
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

In [ ]:
#Amostra

vaccines.tail()

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
360,2021-12-27,Brazil,215313504,329011365,165952037,0.7707,142764283,0.6631,25218893,0.1171,2021-12,2021
361,2021-12-28,Brazil,215313504,329861730,166062249,0.7713,142965728,0.6640,25758909,0.1196,2021-12,2021
362,2021-12-29,Brazil,215313504,330718457,166143380,0.7716,143282084,0.6655,26219623,0.1218,2021-12,2021
363,2021-12-30,Brazil,215313504,331164041,166185628,0.7718,143398692,0.6660,26507937,0.1231,2021-12,2021
364,2021-12-31,Brazil,215313504,331273910,166195505,0.7719,143436012,0.6662,26571077,0.1234,2021-12,2021


In [ ]:
#Carregamento do arquivo em disco local para posterior upload na plataforma Looker Studio.

vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

# 3. Exploração Interativa dos Dados

Nessa etapa final, elucida-se quais são os indicadores alimentados e apresentados no Dashboard a partir dos arquivos (casos e vacinação) gerados pelo processamento das bases de dados.

KPI ou indicadores chave de desempenho, que são:

- Casos e mortes nas 24 horas;
- Média móvel (7 dias) de casos e mortes;
- Tendência de casos e mortes;
- Proporção de vacinados com 1ª, 2ª e 3ª doses.

EDA ou análise exploratória dos dados por meio de gráficos, que são:

- Distribuição do números de casos e mortes ao longo do tempo;
-Distribuição da média móvel (7 dias) do números de casos e mortes ao longo do tempo;
-Distribuição geográfica dos casos por estado por dia.

Novamente, para acessar o Dashboard acesse o [link](https://lookerstudio.google.com/reporting/c9b99be6-9ccf-45df-afd4-91dfa8225f8d/page/zM9xD).